## Read Scoring matrix
The example from Durbin, et al., used BLOSUM50.

In [1]:
gap_score = -8
aa_pairs = dict()
f = open('BLOSUM50.txt','r')
for line in f:
    if line.startswith('#'):
        continue
    elif line.startswith(' '):
        headers = line.strip().split()
    else:
        tmp_list = line.strip().split()
        aa_pairs[tmp_list[0]] = dict()
        for i in range(1,len(tmp_list)):
            aa_pairs[tmp_list[0]][headers[i-1]] = int(tmp_list[i])
f.close()

## Functions for printing results

In [59]:
def print_matrix(tmp_matrix, tmp_dir_list):
    size_x, size_y = tmp_matrix.shape
    for i in range(size_x):
        out_str = []
        for j in range(size_y):
            out_str.append('%3s'%tmp_dir_list[(i,j)])
        print(" ".join(out_str))
    print(tmp_matrix)

def print_aligned(tmp_path, tmp_seq1, tmp_seq2, tmp_pos1, tmp_pos2):
    aligned1 = []
    aligned2 = []
    pos1 = tmp_pos1-tmp_path.count('d')-tmp_path.count('v')
    pos2 = tmp_pos2-tmp_path.count('d')-tmp_path.count('h')
    for i in range(len(tmp_path)):
        if tmp_path[i] == 'd':
            aligned1.append(tmp_seq1[pos1])
            aligned2.append(tmp_seq2[pos2])
            pos1 += 1
            pos2 += 1
        elif tmp_path[i] == 'v':
            aligned1.append(tmp_seq1[pos1])
            aligned2.append('-')
            pos1 += 1
        elif tmp_path[i] == 'h':
            aligned1.append('-')
            aligned2.append(tmp_seq2[pos2])
            pos2 += 1
    print("%s\n%s"%( ''.join(aligned1), ''.join(aligned2)))

In [63]:
import numpy as np

def global_alignment(tmp_matrix, tmp_pos1, tmp_pos2, tmp_chr1, tmp_chr2):    
    if tmp_pos1 == 0 and tmp_pos2 == 0:
        return 's',0
    elif tmp_pos2 == 0:
        return 'v',tmp_matrix[tmp_pos1-1][0] + gap_score
    elif tmp_pos1 == 0:
        return 'h',tmp_matrix[0][tmp_pos2-1] + gap_score
    else:
        d_score = tmp_matrix[tmp_pos1-1][tmp_pos2-1]
        d_score += aa_pairs[tmp_chr1][tmp_chr2]
        v_score = tmp_matrix[tmp_pos1-1][tmp_pos2] + gap_score
        h_score = tmp_matrix[tmp_pos1][tmp_pos2-1] + gap_score
        
        max_score = max([d_score,v_score,h_score])
        tmp_dir_list = []
        if d_score == max_score:
            tmp_dir_list.append('d')
        if v_score == max_score:
            tmp_dir_list.append('v')
        if h_score == max_score:
            tmp_dir_list.append('h')
            
        return ''.join(tmp_dir_list), max_score

def local_alignment(tmp_matrix, tmp_pos1, tmp_pos2, tmp_chr1, tmp_chr2):    
    if tmp_pos1 == 0 or tmp_pos2 == 0:
        return 's',0
    else:
        d_score = tmp_matrix[tmp_pos1-1][tmp_pos2-1]
        d_score += aa_pairs[tmp_chr1][tmp_chr2]
        v_score = tmp_matrix[tmp_pos1-1][tmp_pos2] + gap_score
        h_score = tmp_matrix[tmp_pos1][tmp_pos2-1] + gap_score
        
        max_score = max([d_score,v_score,h_score,0])
        tmp_dir_list = []
        if d_score == max_score:
            tmp_dir_list.append('d')
        if v_score == max_score:
            tmp_dir_list.append('v')
        if h_score == max_score:
            tmp_dir_list.append('h')
        if len(tmp_dir_list) == 0:
            tmp_dir_list.append('s')
            
        return ''.join(tmp_dir_list), max_score

def find_path(tmp_dir_list, tmp_pos1, tmp_pos2):
    if not (tmp_pos1, tmp_pos2) in tmp_dir_list:
        return []
    
    tmp_dir = tmp_dir_list[(tmp_pos1, tmp_pos2)]
    ## Preferred to be matched if there are multiple paths
    if tmp_dir == 's':
        return []
    if tmp_dir.find('d') >= 0:
        tmp_dir = 'd'
        next_pos1 = tmp_pos1-1
        next_pos2 = tmp_pos2-1
    elif tmp_dir == 'v':
        next_pos1 = tmp_pos1-1
        next_pos2 = tmp_pos2
    elif tmp_dir == 'h':
        next_pos1 = tmp_pos1
        next_pos2 = tmp_pos2-1
    else:
        return [tmp_dir]
    
    return [tmp_dir]+find_path(tmp_dir_list, next_pos1, next_pos2)
    
def align_pairwise(tmp_seq1, tmp_seq2):
    len_seq1 = len(tmp_seq1)
    len_seq2 = len(tmp_seq2)
    
    dir_list = dict()
    max_score = -100
    (max_pos1, max_pos2) = (0,0)
    score_matrix = np.zeros( (len_seq1+1, len_seq2+1) )
    for pos1 in range(len_seq1+1):
        chr1 = tmp_seq1[pos1-1]
        for pos2 in range(len_seq2+1):
            chr2 = tmp_seq2[pos2-1]
            ## for global alignment
            #tmp_dir, tmp_score = global_alignment(score_matrix, pos1, pos2, chr1, chr2)
            ## for local alignment
            tmp_dir, tmp_score = local_alignment(score_matrix, pos1, pos2, chr1, chr2)
            
            score_matrix[pos1][pos2] = tmp_score
            dir_list[(pos1,pos2)] = tmp_dir
            if max_score < tmp_score:
                max_score = tmp_score
                max_pos1 = pos1
                max_pos2 = pos2
    
    ## for global alignment
    #max_pos1 = pos1
    #max_pos2 = pos2
    
    print_matrix( score_matrix, dir_list )
    path_str = find_path(dir_list, max_pos1, max_pos2)
    print(path_str[::-1])
    return max_score, path_str[::-1], max_pos1, max_pos2

seq_1 = 'PAWHEAE'
seq_2 = 'HEAGAWGHEE'

max_score, max_path, max_pos1, max_pos2 = align_pairwise(seq_1,seq_2)
print("Max score: %d"%(max_score))
print_aligned(max_path, seq_1, seq_2, max_pos1, max_pos2)

  s   s   s   s   s   s   s   s   s   s   s
  s   s   s   s   s   s   s   s   s   s   s
  s   s   s   d   d   d   s   d   s   s   s
  s   s   s   s   d   s   d   h   h   s   s
  s   d   h   s   s   d   v   d   d   h   h
  s   v   d   h   h   s   v   v   d   d  dh
  s   s   v   d   h  dh   s   d   v   v   d
  s   d   d   v   d   d   h   s   d   d   d
[[  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   5.   0.   5.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   2.   0.  20.  12.   4.   0.   0.]
 [  0.  10.   2.   0.   0.   0.  12.  18.  22.  14.   6.]
 [  0.   2.  16.   8.   0.   0.   4.  10.  18.  28.  20.]
 [  0.   0.   8.  21.  13.   5.   0.   4.  10.  20.  27.]
 [  0.   0.   6.  13.  18.  12.   4.   0.   4.  16.  26.]]
['d', 'd', 'h', 'd', 'd']
Max score: 28
AW-HE
AWGHE
